In [1]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import pandas as pd
import json
import csv
import sys
import time

ckey = 'bXRPnBdpsgRdrkfbrHBHFInjl'
csecret = 'xm93rCpGvtV6E2lwx6392l5uUfYpz9dgCe7zREGtCWeWd5b6WB'
atoken = '1191115298788777987-qUAhRZ5dAw2wN1QewZxjROvtnVLHBs'
asecret = '595HhAG8XXlbN7BPYfLVUio1ozLqmLMnFDDIaHfzPmeSv'

def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet.id for tweet in tweets]
    DataSet['tweetText'] = [tweet.text for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    DataSet['userID'] = [tweet.user.id for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets]
    DataSet['Language'] = [tweet.user.lang for tweet in tweets]
    tweets_place= []
    #users_retweeted = []
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    DataSet['TweetPlace'] = [i for i in tweets_place]
    #DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]

    return DataSet

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret,'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#auth = tweepy.AppAuthHandler('key', 'key')

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)
else:
# I am trying to download from Dec 1st to Dec 7th but I am not able to

    cursor = tweepy.Cursor(api.search, q='#RestoreTheSoulOfAmerica',lang='en')
    results=[]
    for item in cursor.items(10000):
        results.append(item)

    DataSet = toDataFrame(results)

In [2]:
DataSet["text"] = DataSet["tweetText"]
DataSet.shape

(6, 20)

In [3]:
import re
import preprocessor as p

DataSet["text"] = DataSet["tweetText"]

pd.options.display.max_colwidth = 500

def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

DataSet["text"]  = DataSet.apply(preprocess_tweet, axis=1)

DataSet["text"] = DataSet["text"].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


DataSet[["tweetText", "text"]].head()

,tweetText,text
0,Too much?? 🤷🏻‍♀️#dumptrump #BidenHarris2020 #restorethesoulofamerica https://t.co/od7ADPY6iP,too much
1,"RT @uhactually: At Integralists for Biden, we’re pleased to announce Martin O’Malley as national co-chair. #CommonGoodConservatism #Restore…",at integralists for biden were pleased to announce martin omalley as national co chair
2,RT @mark_vallen: Liar &amp; plagiarist #JoeBiden was forced to END his 1988 campaign for Pres. when caught STEALING lines from speeches by Pres…,liar amp plagiarist was forced to end his campaign for pres when caught stealing lines from speeches by pres
3,Liar &amp; plagiarist #JoeBiden was forced to END his 1988 campaign for Pres. when caught STEALING lines from speeches… https://t.co/NDODfh1NZV,liar amp plagiarist was forced to end his campaign for pres when caught stealing lines from speeches
4,I pledge to support the #President that wins the next #election of these beautiful #UnitedStatesofAmerica 🗽Who has… https://t.co/AeFop7Ftx6,i pledge to support the that wins the next of these beautiful who has


In [4]:
from nltk.tokenize import word_tokenize



def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
DataSet['text2'] = DataSet['text'].apply(custom_tokenize)


DataSet[["text", "text2"]].head()


,text,text2
0,too much,"[too, much]"
1,at integralists for biden were pleased to announce martin omalley as national co chair,"[at, integralists, for, biden, were, pleased, to, announce, martin, omalley, as, national, co, chair]"
2,liar amp plagiarist was forced to end his campaign for pres when caught stealing lines from speeches by pres,"[liar, amp, plagiarist, was, forced, to, end, his, campaign, for, pres, when, caught, stealing, lines, from, speeches, by, pres]"
3,liar amp plagiarist was forced to end his campaign for pres when caught stealing lines from speeches,"[liar, amp, plagiarist, was, forced, to, end, his, campaign, for, pres, when, caught, stealing, lines, from, speeches]"
4,i pledge to support the that wins the next of these beautiful who has,"[i, pledge, to, support, the, that, wins, the, next, of, these, beautiful, who, has]"


In [5]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

DataSet['text3'] = DataSet['text2'].apply(lambda x: [stemmer.stem(y) for y in x]) 


DataSet[["tweetText", "text2", "text3"]].head()

,tweetText,text2,text3
0,Too much?? 🤷🏻‍♀️#dumptrump #BidenHarris2020 #restorethesoulofamerica https://t.co/od7ADPY6iP,"[too, much]","[too, much]"
1,"RT @uhactually: At Integralists for Biden, we’re pleased to announce Martin O’Malley as national co-chair. #CommonGoodConservatism #Restore…","[at, integralists, for, biden, were, pleased, to, announce, martin, omalley, as, national, co, chair]","[at, integralist, for, biden, were, pleas, to, announc, martin, omalley, as, nation, co, chair]"
2,RT @mark_vallen: Liar &amp; plagiarist #JoeBiden was forced to END his 1988 campaign for Pres. when caught STEALING lines from speeches by Pres…,"[liar, amp, plagiarist, was, forced, to, end, his, campaign, for, pres, when, caught, stealing, lines, from, speeches, by, pres]","[liar, amp, plagiarist, was, forc, to, end, his, campaign, for, pres, when, caught, steal, line, from, speech, by, pres]"
3,Liar &amp; plagiarist #JoeBiden was forced to END his 1988 campaign for Pres. when caught STEALING lines from speeches… https://t.co/NDODfh1NZV,"[liar, amp, plagiarist, was, forced, to, end, his, campaign, for, pres, when, caught, stealing, lines, from, speeches]","[liar, amp, plagiarist, was, forc, to, end, his, campaign, for, pres, when, caught, steal, line, from, speech]"
4,I pledge to support the #President that wins the next #election of these beautiful #UnitedStatesofAmerica 🗽Who has… https://t.co/AeFop7Ftx6,"[i, pledge, to, support, the, that, wins, the, next, of, these, beautiful, who, has]","[i, pledg, to, support, the, that, win, the, next, of, these, beauti, who, has]"


In [6]:
import datetime
year = datetime.date.today().year
month = datetime.date.today().month
month = '{:02d}'.format(month)
day = datetime.date.today().day
day = '{:02d}'.format(day)

today = datetime.date.today().strftime("%Y.%m.%d")
DataSet["keresoszo"] = "#RestoreTheSoulOfAmerica"
DataSet["letoltes_datum"] = today

In [7]:
DataSet.to_csv('data/RestoreTheSoulOfAmerica' + str(year) + str(month) + str(day) + '.csv',index=False)

In [8]:
DataSet.head()

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,userScreen,userName,userCreateDt,...,userTimezone,Coordinates,GeoEnabled,Language,TweetPlace,text,text2,text3,keresoszo,letoltes_datum
0,1311753988786843650,Too much?? 🤷🏻‍♀️#dumptrump #BidenHarris2020 #restorethesoulofamerica https://t.co/od7ADPY6iP,0,0,Twitter for iPhone,2020-10-01 19:44:56,16796529,michelle_ad,Michelle Adcock 🐝💛💙🐝,2008-10-15 23:13:42,...,None,None,True,None,null,too much,"[too, much]","[too, much]",#RestoreTheSoulOfAmerica,2020.10.03
1,1311077329419018241,"RT @uhactually: At Integralists for Biden, we’re pleased to announce Martin O’Malley as national co-chair. #CommonGoodConservatism #Restore…",1,0,Twitter for iPhone,2020-09-29 22:56:08,347576155,jacqueportland,Jacque Dewolf,2011-08-03 00:55:44,...,None,None,False,None,null,at integralists for biden were pleased to announce martin omalley as national co chair,"[at, integralists, for, biden, were, pleased, to, announce, martin, omalley, as, national, co, chair]","[at, integralist, for, biden, were, pleas, to, announc, martin, omalley, as, nation, co, chair]",#RestoreTheSoulOfAmerica,2020.10.03
2,1310835996011896832,RT @mark_vallen: Liar &amp; plagiarist #JoeBiden was forced to END his 1988 campaign for Pres. when caught STEALING lines from speeches by Pres…,1,0,Twitter for Android,2020-09-29 06:57:10,1272293552895098881,JParsival,PjSinatra,2020-06-14 22:23:43,...,None,None,False,None,null,liar amp plagiarist was forced to end his campaign for pres when caught stealing lines from speeches by pres,"[liar, amp, plagiarist, was, forced, to, end, his, campaign, for, pres, when, caught, stealing, lines, from, speeches, by, pres]","[liar, amp, plagiarist, was, forc, to, end, his, campaign, for, pres, when, caught, steal, line, from, speech, by, pres]",#RestoreTheSoulOfAmerica,2020.10.03
3,1310758055525535745,Liar &amp; plagiarist #JoeBiden was forced to END his 1988 campaign for Pres. when caught STEALING lines from speeches… https://t.co/NDODfh1NZV,1,0,Twitter Web App,2020-09-29 01:47:27,2990624796,mark_vallen,Mark Vallen,2015-01-20 04:36:54,...,None,None,False,None,null,liar amp plagiarist was forced to end his campaign for pres when caught stealing lines from speeches,"[liar, amp, plagiarist, was, forced, to, end, his, campaign, for, pres, when, caught, stealing, lines, from, speeches]","[liar, amp, plagiarist, was, forc, to, end, his, campaign, for, pres, when, caught, steal, line, from, speech]",#RestoreTheSoulOfAmerica,2020.10.03
4,1310232735441735680,I pledge to support the #President that wins the next #election of these beautiful #UnitedStatesofAmerica 🗽Who has… https://t.co/AeFop7Ftx6,0,0,Twitter for iPhone,2020-09-27 15:00:01,19007920,EdMermelstein,Edward Mermelstein,2009-01-15 02:50:36,...,None,None,True,None,null,i pledge to support the that wins the next of these beautiful who has,"[i, pledge, to, support, the, that, wins, the, next, of, these, beautiful, who, has]","[i, pledg, to, support, the, that, win, the, next, of, these, beauti, who, has]",#RestoreTheSoulOfAmerica,2020.10.03
